# Prototype 1

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Expression, SolverFactory, NonNegativeReals, minimize

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(bounds=(0, 10), within=NonNegativeReals)
model.y = Var(bounds=(0, 10), within=NonNegativeReals)

# Define objectives
model.obj1 = Expression(expr=model.x + 2 * model.y)
model.obj2 = Expression(expr=2 * model.x + model.y)

# Define constraints
model.con1 = Constraint(expr=model.x + model.y >= 5)
model.con2 = Constraint(expr=model.x - model.y <= 3)

# Activate only one objective at a time
# model.dummy_obj = Objective(expr=0, sense=minimize)  # Dummy objective to activate obj1

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Display results
# model.display()

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

## Scalarize Method

In [2]:
from pyomo.environ import Suffix

# Define a suffix to store the objective expressions and their weights
model.objectives = Suffix(direction=Suffix.IMPORT)
model.objectives.update([(model.obj1, 0.5), (model.obj2, 0.5)])

# Set the objective to minimize the weighted sum of the objectives
model.scalarize_objective = Objective(expr=sum(obj * weight for obj, weight in enumerate(model.objectives)), sense=minimize)

# Solve the model again with the new objective
results = solver.solve(model, tee=True)

model.pprint()

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [3]:
model.objectives.pprint()

objectives : Direction=IMPORT, Datatype=FLOAT
    Key : Value


In [7]:
for i, obj, weight in enumerate(model.objectives.items()):
    print(f"Objective {obj}: {model.objectives[obj]} with weight {weight}")